In [2]:
import gurobipy as gb
import random 


## 0-1 Knapsack Inequalities
**Ref. Chapter 9.3 Wolsey, Integer Programming**

Consider the set $ X = \{x \in \{0,1\}^n : \sum_{j=1}^{n} a_j x_j \le b\} $.  W.l.o.g., we  assume that $b>0$ and $a_j > 0$. In fact, if there exists an index $j$ such that  $a_j < 0$ we can complement the corresponding variable $\bar x_j = 1 - x_j$ and obtain all positive coefficients and $\bar b = b - a_j > 0$.  

## Cover inequalities

### Definition 
A set $C \subseteq N$ is called a *cover* of $X$ if it satisfies $\sum_{j \in C} a_j > b$. 

A cover $C$ is *minimal* if $C \setminus \{j\}$ is not a cover for **any** $j \in C$.

Let $x_C$ be the *incidence vector* of $C$, i.e., a vector with the $j$-th component  equal to 1 if $j \in C$, 0 otherwise.

### Cover inequality
> If a set $C$ is a cover of $X$, then the *cover inequality* 
> $$\sum_{j \in C} x_j \le |C| -1$$
> is valid for ${conv}(X)$

#### Proof

> We show that a point $x$ that does not satisy the cover inequality does not belong to $X$.
> Choose a set $R \subseteq N$ and define a vector $x^R \in \{0,1\}^n $ such that $x_j = 1$ if $j \in R$ and 0 otherwise. 

>Suppose that $x^R$ does not satisfy $\sum_{j \in C} x_j \le |C| -1$, i.e., $\sum_{j \in C} x^R_j > |C| - 1 $. 

> If $\sum_{j \in C} x_j^R > |C| -1$ , then $|R \cap C| = |C|$, that is, $C \subseteq R$. 
> By definition, $$\sum_{j=1}^n a_j x_j^R = \sum_{j \in R} a_j \ge \sum_{j \in C} a_j > b, $$

> thus, $x^R \not\in X$. 

### Example
Consider the knapsack set:
$$ X = \{12 x_1 + 9x_2 + 8x_3 + 6x_4 + 6x_5 + 5 x_6 + 3x_7 \le 22, x\in \{0,1\} ^ 7\}$$

**Minimal cover inequalities** for $conv(X)$:
$$ 
x_1 + x_2 + x_3 \le 2 \\
x_1 + x_2 + x_6 \le 2 \\
x_1 + x_5 + x_6 \le 2 \\
x_3 + x_4 + x_5 + x_6 \le 3
$$


## Separation

Consider the following problem:

$$ \max 15x_1 +11x_2 +9x_3 +6x_4 +5x_5 +4x_6 + x_7 \\ 
12 x_1 + 9x_2 + 8x_3 + 6x_4 + 6x_5 + 5 x_6 + 3x_7 \le 22\\
x \in [0,1]^7 $$



In [4]:
n = 7
c = {1:15, 2:11, 3:9, 4:6, 5:5, 6:4, 7:1}
a = {1:12, 2:9, 3:8, 4:6, 5:6, 6:5, 7:3}
b = 22

knapsackLP = gb.Model('knapsack')

x = knapsackLP.addVars(c.keys(),vtype=gb.GRB.CONTINUOUS, \
                       ub=1.0, name='x',\
                       obj=c)

knapsackLP.ModelSense = gb.GRB.MAXIMIZE

knapsackLP.addConstr(x.prod(a) <= b, name='c')

knapsackLP.update()

knapsackLP.write('knapsack.lp')

knapsackLP.optimize()

Optimize a model with 1 rows, 7 columns and 7 nonzeros
Coefficient statistics:
  Matrix range     [3e+00, 1e+01]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+01]
Presolve time: 0.01s
Presolved: 1 rows, 7 columns, 7 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.7500000e+01   8.333333e-01   0.000000e+00      0s
       1    2.7125000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.03 seconds
Optimal objective  2.712500000e+01


In [6]:
LPsol = knapsackLP.getAttr('x', x)
LPsol 

{1: 1.0, 2: 1.0, 3: 0.125, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0}

In [10]:
x[1].x

1.0

### Separation problem

> Given a nonintegral point $x^*$ with $0 \le x^* \le 1$ for all $j \in N$, 
> the *separation problem* calls for finding a violated cover inequalities or prove that none exists.  

A cover inequality $\sum_{j \in C} x_j \le |C| -1$ can be rewritten as $\sum_{j \in C}(1 - x_j) \ge 1$. Thus, such an inequality is violated by $x^*$ if 

$$
\sum_{j \in C} (1-x^*_j) < 1
$$

Define the following *separation problem*:

$$
\chi = \min_{C \subseteq N} \{\sum_{j \in C} (1-x^*_j): C \text{ is a cover of } X  \}
$$

If $\chi < 1$ then there exists a violated cover inequality, otherwise such an inequality does not exist.

The separation problem can be restated as:

$$
\chi = \min \{\sum_{j \in C} (1-x^*_j): {C \subseteq N}, \sum_{j \in C} a_j > b\}
$$

that is, by introducing binary variables $z_j$ with the meaning 

$$
z_j = \begin{cases}
1 \text { if } j \text{ is in the cover,}\\
0 \text { otherwise.}
\end{cases}
$$

one has

$$
\min {\sum_{j=1}^n (1 - x^*_j)z_j }\\
\text{s.t. } \sum_{j=1}^n a_j z_j \ge b +1 \\
z \in \{0,1\}^n
$$






In our example:

In [14]:
ncover = 0

In [26]:
sepacover = gb.Model('sep')

sepaobj = {key:1.0 - val for key,val in LPsol.items()}

sepaobj

{1: 0.0,
 2: 0.09090909090909094,
 3: 0.9090909090909091,
 4: 0.9090909090909091,
 5: 0.9090909090909091,
 6: 1.0,
 7: 1.0}

In [27]:
z = sepacover.addVars(LPsol.keys(),vtype=gb.GRB.BINARY, \
                      name='z',\
                      obj=sepaobj)

sepacover.addConstr(z.prod(a) >= b + 1, name='c')

sepacover.update()

sepacover.write('sepacover.lp')

sepacover.optimize()


sepasol = sepacover.getAttr('x', z)

sepasol


Optimize a model with 1 rows, 7 columns and 7 nonzeros
Variable types: 0 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [3e+00, 1e+01]
  Objective range  [9e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+01]
Found heuristic solution: objective 1.9090909
Presolve removed 1 rows and 7 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 4 available processors)

Solution count 2: 1 1.90909 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+00, best bound 1.000000000000e+00, gap 0.0000%


{1: 1.0, 2: 1.0, 3: 1.0, 4: 0.0, 5: -0.0, 6: 0.0, 7: 0.0}

In [28]:
print ('Separation problem optimal value:', \
       sepacover.ObjVal)


if sepacover.ObjVal < 1:
    ncover += 1
    print ('Found violated cover inequality:', end=' ') 
    coversize = 0
    coverineq = {}
    for i in sepasol:
        if sepasol[i] > 0.001:
            coversize += 1
            coverineq[i] = 1.0
    print
    print ('Cover:', coverineq, 'Rhs:', coversize - 1)


Separation problem optimal value: 1.0


<div class="alert alert-success">
The violated inequality found is **added** to the original problem.
</div>
<div class="alert alert-danger">
If no violated inequality exists, **stop**
</div>

In [25]:
knapsackLP.addConstr(x.prod(coverineq) \
                     <= coversize - 1, \
                     name='Cover_%d' % ncover)

knapsackLP.update()
knapsackLP.write('knapsack.lp')

knapsackLP.optimize()

LPsol = knapsackLP.getAttr('x', x)
LPsol

Optimize a model with 4 rows, 7 columns and 16 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+01]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.6833333e+01   1.666667e-01   0.000000e+00      0s
       1    2.6818182e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds
Optimal objective  2.681818182e+01


{1: 1.0,
 2: 0.9090909090909091,
 3: 0.09090909090909094,
 4: 0.09090909090909091,
 5: 0.09090909090909094,
 6: 0.0,
 7: 0.0}